In [1]:
import urllib.request
import pandas as pd
import zipfile

In [9]:
#GP Practice data ingestion

url = 'https://files.digital.nhs.uk/assets/ods/current/epraccur.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
first_file = zip_file_object.namelist()[0]
file = zip_file_object.open(first_file)
content = file.read()
csv_file = open('data/epraccur_data.csv', 'wb')
csv_file.write(content)
csv_file.close()
header_list = ["Organisation Code", "Name", "National Grouping", "High Level Health Geography", "Address line 1", "Address line 2", "Address line 3", 
"Address line 4", "Address line 5","Postcode","Open Date","Close Date","Status Code","Organisation Sub-Type Code","Commissioner","Join Provider/Purchaser Date",
"Left Provider/Purchaser Date","Contact Telephone Number", "Null 1", "Null 2", "Null 3", "Amended Record Indicator", "Null 4", "Provider/Purchaser",
"Null 5", "Prescribing Setting", "Null 6"]

In [71]:
gp_practice_df = pd.read_csv('data/epraccur_data.csv', names=header_list)
gp_practice_df = gp_practice_df[(gp_practice_df["Status Code"] == "A") & (gp_practice_df["Prescribing Setting"] == 4) & (gp_practice_df["Postcode"].str.contains('CB1 |CB2 |CB3 |CB4 |CB5 '))].reset_index(drop = True)


In [72]:
gp_practice_df['Full Address'] = gp_practice_df[['Address line 2', 'Address line 3', 'Address line 4',]].astype(str).agg(', '.join, axis=1)
gp_practice_df['Full Address'] = gp_practice_df['Full Address'].str.title() + ', ' + gp_practice_df['Postcode']
gp_practice_df['Full Address'] = gp_practice_df['Full Address'].str.replace('Nan', '').str.replace(' ,', ' ')
gp_practice_df['Name'] = gp_practice_df['Name'].str.title()

In [73]:
gp_practice_df_1 = gp_practice_df[['Organisation Code', 'Name', 'Full Address', 'Postcode', 'Contact Telephone Number']]
gp_practice_df_1

,Organisation Code,Name,Full Address,Postcode,Contact Telephone Number
0,D81001,Lensfield Medical Practice,"48 Lensfield Road, Cambridge, Cambridgeshire, ...",CB2 1EH,01223 651020
1,D81002,Huntingdon Road Surgery,"1 Huntingdon Road, Cambridge, CB3 0DB",CB3 0DB,01223 364127
2,D81003,York Street Medical Practice,"146-148 York Street, Cambridge, Cambridgeshire...",CB1 2PY,01223 364116
3,D81005,Newnham Walk Surgery,"Wordsworth Grove, Cambridge, Cambridgeshire, C...",CB3 9HS,01223 366811
4,D81012,Cornford House Surgery,"364 Cherry Hinton Road, Cambridge, Cambridgesh...",CB1 8BA,01223 247505
5,D81013,Trumpington Street Medical Practice,"56 Trumpington Street, Cambridge, Cambridgeshi...",CB2 1RG,01223 361611
6,D81016,Arbury Road Surgery,"114 Arbury Road, Cambridge, Cambridgeshire, CB...",CB4 2JG,01223 364433
7,D81017,Mill Road Surgery,"Cambridge, Cambridgeshire, CB1 3DG",CB1 3DG,01223 247812
8,D81025,Cherry Hinton Medical Centre,"34 Fishers Lane, Cherry Hinton, Cambridge, CB1...",CB1 9HR,01223 652221
9,D81037,Bridge Street Medical Centre,"Passage,Off Bridge St, Cambridge, CB2 3LS",CB2 3LS,01223 652671


In [74]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [75]:
geolocator = Nominatim(user_agent="open_access_nhs")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [76]:
gp_practice_df_1["location"] = gp_practice_df_1["Postcode"].apply(geolocator.geocode)
gp_practice_df_1["Point"] = gp_practice_df_1["location"].apply(lambda loc: tuple(loc.point) if loc else None)
gp_practice_df_1[['Latitude', 'Longitude', 'Altitude']] = gp_practice_df_1['Point'].to_list()


C:\Users\MATTIA~1.FIC\AppData\Local\Temp/ipykernel_18020/2279169664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp_practice_df_1["location"] = gp_practice_df_1["Postcode"].apply(geolocator.geocode)
C:\Users\MATTIA~1.FIC\AppData\Local\Temp/ipykernel_18020/2279169664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gp_practice_df_1["Point"] = gp_practice_df_1["location"].apply(lambda loc: tuple(loc.point) if loc else None)
C:\Users\MATTIA~1.FIC\AppData\Local\Temp/ipykernel_18020/2279169664.py:3: Se

In [77]:
gp_practice_df_2 = gp_practice_df_1[["Organisation Code","Name", "Full Address", "Postcode", "Contact Telephone Number", "Latitude", "Longitude"]]
gp_practice_df_2.to_csv('data/cambridge_gp_practices.csv')

In [78]:
import folium

In [101]:
frame = folium.Figure(width=500, height=500)
fig = folium.Map(
    location=[52.2, 0.12],
    tiles="cartodbpositron",
    zoom_start=12).add_to(frame)
for code, name, address, number, lat, lon in zip(gp_practice_df_2['Organisation Code'],
gp_practice_df_2['Name'],
gp_practice_df_2['Full Address'],
gp_practice_df_2['Contact Telephone Number'],
gp_practice_df_2['Latitude'],
gp_practice_df_2['Longitude']):
    folium.Marker(
        [lat, lon],
        popup = folium.Popup('<b>' + 'Name: ' + '</b>'  + str(name) + '<br>'
        '<b>' + 'GP Practice Code: ' + '</b>' + str(code) + '<br>'
        '<b>' + 'Address: ' + '</b>' + str(address) + '<br>'
        '<b>' + 'Telephone Number: ' + '</b>' + str(number) + '<br>',
        max_width=len(address)*20),
        icon=folium.Icon(color="darkblue", prefix='fa', icon="fa-medkit"),
        ).add_to(fig)
fig